<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment03/seq2seq_without_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/lexicons.zip'

Archive:  /content/drive/MyDrive/lexicons.zip
   creating: lexicons/
  inflating: lexicons/ta.translit.sampled.test.tsv  
  inflating: lexicons/ta.translit.sampled.dev.tsv  
  inflating: lexicons/ta.translit.sampled.train.tsv  


In [ ]:
!pip install wandb

     |████████████████████████████████| 2.1MB 8.1MB/s 
     |████████████████████████████████| 133kB 49.2MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 163kB 33.1MB/s 
     |████████████████████████████████| 71kB 12.4MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=9773f305de558708b1207e51d2786e61c9a8a569c12313328b54f180ed6991b3
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=7d59fb3ef938b8c7a4149f9779339f9de8d611721f3f7c9ebf4b84e7e6c99e9b
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from wandb.keras import WandbCallback

In [ ]:
train_df = pd.read_csv("/content/lexicons/ta.translit.sampled.train.tsv", sep = '\t', header = None)
train_df = train_df.dropna(axis=0)

val_df = pd.read_csv("/content/lexicons/ta.translit.sampled.dev.tsv", sep = '\t', header = None)
val_df = val_df.dropna(axis=0)

In [ ]:
train_df

,0,1,2
0,ஃபியட்,fiat,2
1,ஃபியட்,phiyat,1
2,ஃபியட்,piyat,1
3,ஃபிரான்ஸ்,firaans,1
4,ஃபிரான்ஸ்,france,2
...,...,...,...
68213,ஹோல்ட்,holtt,1
68214,ஹோல்ட்,hoold,1
68215,ஹோல்ட்,hoolt,1
68216,ஹோல்ட்,hooltt,1


In [ ]:
val_df

,0,1,2
0,ஃபயர்,fire,3
1,ஃபயர்,phayar,1
2,ஃபார்,baar,1
3,ஃபார்,bar,1
4,ஃபார்,far,1
...,...,...,...
6822,ஹோலி,hoolley,1
6823,ஹோலி,hoolli,2
6824,ஹோலி,hoolly,1
6825,ஹோலி,hooly,2


In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for input_text,target_text in zip(train_df[1][0:],train_df[0][0:]):
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

if ' ' not in input_token_index:
  input_token_index[' '] = len(input_token_index)

if ' ' not in target_token_index:
  target_token_index[' '] = len(target_token_index)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) + 1
num_decoder_tokens = len(target_characters) + 1
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :] = target_token_index[" "]
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 68215
Number of unique input tokens: 27
Number of unique output tokens: 47
Max sequence length for inputs: 30
Max sequence length for outputs: 26


In [ ]:
# Vectorize the data.
val_input_texts = []
val_target_texts = []

for val_input_text,val_target_text in zip(val_df[1][0:],val_df[0][0:]):
    val_input_texts.append(val_input_text)
    val_target_texts.append(val_target_text)

val_encoder_input_data = np.zeros(
    (len(val_input_texts), max_encoder_seq_length), dtype="float32"
)
val_decoder_input_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length), dtype="float32"
)
val_decoder_target_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(val_input_text):
        val_encoder_input_data[i, t] = input_token_index[char]
    encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(val_target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :] = target_token_index[" "]
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
def model(cell_name, embedding_size, hidden_size, num_enc_layers, num_dec_layers, dropout, r_dropout, batch_size, num_epochs, optimizer):
  
  cell_type = { "RNN": keras.layers.SimpleRNN,
                "GRU": keras.layers.GRU,
                "LSTM": keras.layers.LSTM
  }

  learning_rate = 1e-3
  decay_rate = learning_rate / num_epochs

  optimizers = {"adam": tf.keras.optimizers.Adam(lr=learning_rate, decay=decay_rate),
                "nadam": tf.keras.optimizers.Nadam(lr=learning_rate, decay=decay_rate),
                "rmsprop": tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay_rate),
                "adagrad": tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
  }

  # Define an input sequence and process it.
  encoder_inputs = keras.layers.Input(shape=(None,), name="input_1")
  embedding = keras.layers.Embedding(num_encoder_tokens, embedding_size, name="embedding_1")(encoder_inputs)
  encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_1") (embedding)
  for i in range(1, num_enc_layers):
    encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_"+str(i+1), dropout=dropout,
                                                       recurrent_dropout=r_dropout) (encoder_seq)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None,), name="input_2")
  decoder_embedding = keras.layers.Embedding(num_decoder_tokens, embedding_size, name="embedding_2")(decoder_inputs)

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_seq, *_ = cell_type[cell_name](hidden_size,return_sequences=True, return_state=True, name="decoder_1")(decoder_embedding,initial_state=encoder_state)
  for i in range(1, num_dec_layers):
    decoder_seq, *_ = cell_type[cell_name](hidden_size, return_sequences=True, return_state=True, name="decoder_"+str(i+1),dropout=dropout,
                                      recurrent_dropout=r_dropout) (decoder_seq)

  decoder_outputs = keras.layers.Dense(num_decoder_tokens, activation="softmax", name="dense_1")(decoder_seq)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(
    optimizer=optimizers[optimizer], loss="categorical_crossentropy", metrics=["accuracy"]
  )
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=batch_size,
      epochs=num_epochs,
      validation_data=([encoder_input_data, decoder_input_data],
                      decoder_target_data),
      callbacks=[WandbCallback()]
  )

In [ ]:
def swp():

  hyperparameter_defaults = dict(
      cell_name = "LSTM",
      embedding_size = 256,
      hidden_size = 32,
      num_enc_layers = 2,
      num_dec_layers = 2,
      dropout = 0.2,
      r_dropout = 0.2,
      batch_size = 512,
      num_epochs = 10,
      optimizer = "adam"
  )

  wandb.init(project="Assignment 3 without Attention", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "{}_cell_{}_embSize_{}_hiddenSize_{}_encLayers_{}_decLayers_{}_dropout_{}_rDropout_{}_batchSize_{}_epochs_{}_opt".format(config.cell_name, 
                    config.embedding_size,config.hidden_size, config.num_enc_layers, config.num_dec_layers, config.dropout, config.r_dropout, config.batch_size, config.num_epochs, 
                    config.optimizer)
  
  model(config.cell_name, config.embedding_size,config.hidden_size, config.num_enc_layers, config.num_dec_layers, config.dropout, config.r_dropout, config.batch_size, 
         config.num_epochs, config.optimizer)

In [ ]:
sweep_config = {
  "name": "My Sweep",
  "method": "random",
  "project": "Assignment02",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "cell_name": {
            "values": ['RNN', 'LSTM', 'GRU']
        },
        "embedding_size": {
            "values":[16, 32, 64, 128, 256]
        }, 
        "hidden_size": {
            "values":[16, 32, 64, 128, 256]
        },
        "num_enc_layers":{
            "values":[1, 2, 3]
        },
        "num_dec_layers":{
            "values":[1, 2, 3]
        },  
        "dropout":{
            "values":[0, 0.2, 0.3, 0.4]
        },
        "r_dropout": {
            "values":[0, 0.2, 0.3, 0.4]
        },
        "batch_size": {
            "values":[256, 512, 1024]
        },
        "num_epochs": {
            "values":[5, 10, 15]
        },
        "optimizer": {
            "values":['adam', 'nadam','rmsprop','adagrad']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: do6etd4d
Sweep URL: https://wandb.ai/kodikarthik21/uncategorized/sweeps/do6etd4d


In [ ]:
#wandb.agent('fxc8lkyg', swp)

In [ ]:
def model_without_wandb(cell_name, embedding_size, hidden_size, num_enc_layers, num_dec_layers, dropout, r_dropout, batch_size, num_epochs, optimizer):
  
  cell_type = { "RNN": keras.layers.SimpleRNN,
                "GRU": keras.layers.GRU,
                "LSTM": keras.layers.LSTM
  }

  learning_rate = 1e-3
  decay_rate = learning_rate / num_epochs

  optimizers = {"adam": tf.keras.optimizers.Adam(lr=learning_rate, decay=decay_rate),
                "nadam": tf.keras.optimizers.Nadam(lr=learning_rate, decay=decay_rate),
                "rmsprop": tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay_rate),
                "adagrad": tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
  }

  # Define an input sequence and process it.
  encoder_inputs = keras.layers.Input(shape=(None,), name="input_1")
  embedding = keras.layers.Embedding(num_encoder_tokens, embedding_size, name="embedding_1")(encoder_inputs)
  encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_1") (embedding)
  for i in range(1, num_enc_layers):
    encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_"+str(i+1), dropout=dropout,
                                                       recurrent_dropout=r_dropout) (encoder_seq)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None,), name="input_2")
  decoder_embedding = keras.layers.Embedding(num_decoder_tokens, embedding_size, name="embedding_2")(decoder_inputs)

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_seq, *_ = cell_type[cell_name](hidden_size,return_sequences=True, return_state=True, name="decoder_1")(decoder_embedding,initial_state=encoder_state)
  for i in range(1, num_dec_layers):
    decoder_seq, *_ = cell_type[cell_name](hidden_size, return_sequences=True, return_state=True, name="decoder_"+str(i+1),dropout=dropout,
                                      recurrent_dropout=r_dropout) (decoder_seq)

  decoder_outputs = keras.layers.Dense(num_decoder_tokens, activation="softmax", name="dense_1")(decoder_seq)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(
    optimizer=optimizers[optimizer], loss="categorical_crossentropy", metrics=["accuracy"]
  )
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=batch_size,
      epochs=num_epochs,
      validation_data=([encoder_input_data, decoder_input_data],
                      decoder_target_data)
  )

  return model

In [ ]:
model_without_wandb("LSTM", 256, 256, 2, 2, 0.3, 0, 256, 15, "adam")

Epoch 1/15
267/267 [==============================] - 52s 61ms/step - loss: 1.3203 - accuracy: 0.7062 - val_loss: 0.8467 - val_accuracy: 0.7704
Epoch 2/15
267/267 [==============================] - 15s 56ms/step - loss: 0.8127 - accuracy: 0.7756 - val_loss: 0.7599 - val_accuracy: 0.7901
Epoch 3/15
267/267 [==============================] - 15s 56ms/step - loss: 0.7453 - accuracy: 0.7969 - val_loss: 0.6850 - val_accuracy: 0.8172
Epoch 4/15
267/267 [==============================] - 15s 57ms/step - loss: 0.6806 - accuracy: 0.8196 - val_loss: 0.6300 - val_accuracy: 0.8381
Epoch 5/15
267/267 [==============================] - 15s 56ms/step - loss: 0.6342 - accuracy: 0.8390 - val_loss: 0.5666 - val_accuracy: 0.8639
Epoch 6/15
267/267 [==============================] - 15s 56ms/step - loss: 0.5764 - accuracy: 0.8628 - val_loss: 0.5088 - val_accuracy: 0.8916
Epoch 7/15
267/267 [==============================] - 15s 56ms/step - loss: 0.5132 - accuracy: 0.8897 - val_loss: 0.4524 - val_accuracy:

In [ ]:
model_without_wandb("LSTM", 256, 256, 2, 1, 0.4, 0.2, 256, 15, "nadam")

Epoch 1/15
267/267 [==============================] - 45s 151ms/step - loss: 1.2304 - accuracy: 0.7203 - val_loss: 0.7790 - val_accuracy: 0.7837
Epoch 2/15
267/267 [==============================] - 40s 148ms/step - loss: 0.7612 - accuracy: 0.7910 - val_loss: 0.6738 - val_accuracy: 0.8177
Epoch 3/15
267/267 [==============================] - 39s 147ms/step - loss: 0.6595 - accuracy: 0.8218 - val_loss: 0.5896 - val_accuracy: 0.8443
Epoch 4/15
267/267 [==============================] - 39s 147ms/step - loss: 0.5492 - accuracy: 0.8568 - val_loss: 0.4539 - val_accuracy: 0.8913
Epoch 5/15
267/267 [==============================] - 39s 148ms/step - loss: 0.4499 - accuracy: 0.8945 - val_loss: 0.3874 - val_accuracy: 0.9179
Epoch 6/15
267/267 [==============================] - 40s 150ms/step - loss: 0.3853 - accuracy: 0.9201 - val_loss: 0.3538 - val_accuracy: 0.9363
Epoch 7/15
267/267 [==============================] - 40s 149ms/step - loss: 0.3528 - accuracy: 0.9381 - val_loss: 0.3432 - val_ac

In [ ]:
model_without_wandb("LSTM", 256, 256, 3, 1, 0, 0.2, 256, 15, "rmsprop")

Epoch 1/15
267/267 [==============================] - 66s 225ms/step - loss: 1.1567 - accuracy: 0.7236 - val_loss: 0.7588 - val_accuracy: 0.7923
Epoch 2/15
267/267 [==============================] - 58s 218ms/step - loss: 0.7242 - accuracy: 0.8062 - val_loss: 0.6496 - val_accuracy: 0.8357
Epoch 3/15
267/267 [==============================] - 58s 218ms/step - loss: 0.6503 - accuracy: 0.8398 - val_loss: 0.6205 - val_accuracy: 0.8615
Epoch 4/15
267/267 [==============================] - 58s 219ms/step - loss: 0.6095 - accuracy: 0.8694 - val_loss: 0.6081 - val_accuracy: 0.8895
Epoch 5/15
267/267 [==============================] - 59s 220ms/step - loss: 0.6190 - accuracy: 0.8948 - val_loss: 0.6351 - val_accuracy: 0.9097
Epoch 6/15
267/267 [==============================] - 59s 222ms/step - loss: 0.6438 - accuracy: 0.9151 - val_loss: 0.6585 - val_accuracy: 0.9318
Epoch 7/15
267/267 [==============================] - 59s 220ms/step - loss: 0.6864 - accuracy: 0.9309 - val_loss: 0.7306 - val_ac

In [ ]:
model_without_wandb("GRU", 256, 256, 3, 3, 0, 0, 256, 15, "rmsprop")

Epoch 1/15
267/267 [==============================] - 58s 74ms/step - loss: 1.1920 - accuracy: 0.7170 - val_loss: 0.7567 - val_accuracy: 0.8042
Epoch 2/15
267/267 [==============================] - 19s 70ms/step - loss: 0.7422 - accuracy: 0.8085 - val_loss: 0.7093 - val_accuracy: 0.8234
Epoch 3/15
267/267 [==============================] - 19s 71ms/step - loss: 0.6932 - accuracy: 0.8325 - val_loss: 0.6669 - val_accuracy: 0.8522
Epoch 4/15
267/267 [==============================] - 19s 73ms/step - loss: 0.6742 - accuracy: 0.8545 - val_loss: 0.6561 - val_accuracy: 0.8770
Epoch 5/15
267/267 [==============================] - 19s 73ms/step - loss: 0.6631 - accuracy: 0.8808 - val_loss: 0.6649 - val_accuracy: 0.9002
Epoch 6/15
267/267 [==============================] - 19s 72ms/step - loss: 0.6854 - accuracy: 0.9053 - val_loss: 0.6929 - val_accuracy: 0.9254
Epoch 7/15
267/267 [==============================] - 19s 72ms/step - loss: 0.7356 - accuracy: 0.9245 - val_loss: 0.7716 - val_accuracy:

In [ ]:
model = model_without_wandb("LSTM", 256, 256, 3, 3, 0, 0, 256, 15, "rmsprop")

Epoch 1/15
267/267 [==============================] - 30s 88ms/step - loss: 1.2668 - accuracy: 0.7034 - val_loss: 0.8026 - val_accuracy: 0.7756
Epoch 2/15
267/267 [==============================] - 22s 82ms/step - loss: 0.7979 - accuracy: 0.7792 - val_loss: 0.7734 - val_accuracy: 0.7938
Epoch 3/15
267/267 [==============================] - 22s 81ms/step - loss: 0.7293 - accuracy: 0.8095 - val_loss: 0.7028 - val_accuracy: 0.8240
Epoch 4/15
267/267 [==============================] - 21s 80ms/step - loss: 0.6981 - accuracy: 0.8283 - val_loss: 0.7009 - val_accuracy: 0.8379
Epoch 5/15
267/267 [==============================] - 22s 81ms/step - loss: 0.7095 - accuracy: 0.8418 - val_loss: 0.7171 - val_accuracy: 0.8534
Epoch 6/15
267/267 [==============================] - 22s 82ms/step - loss: 0.7362 - accuracy: 0.8565 - val_loss: 0.7598 - val_accuracy: 0.8676
Epoch 7/15
267/267 [==============================] - 22s 81ms/step - loss: 0.7699 - accuracy: 0.8709 - val_loss: 0.7876 - val_accuracy: